# 240415

## DRF with N:1 Relation

## 댓글 전체조회
1. CommentSerializer 정의
2. url 작성
3. view 함수 작성

In [ ]:
class CommentSerializer(serializers.ModelSerializer):

    class Meta:
        model = Comment
        fields = '__all__'

path('comments/', views.comment_list),

@api_view(['GET'])
def comment_list(request):
    # 전체 댓글 조회
    comments = Comment.objects.all()
    serializer = CommentSerializer(comments, many=True)
    return Response(serializer.data)

## 특정 번호의 댓글 조회
1. url 작성
2. view 함수 작성

In [ ]:
path('comments/<int:comment_pk>/', views.comment_detail),


@api_view(['GET'])
def comment_detail(request, comment_pk):
    comment = Comment.objects.get(pk = comment_pk)
    serializer = CommentSerializer(comment)
    return Response(serializer.data)


## 댓글 생성
1. url 및 view 함수 작성
2. serializer.py 에서 read_only_fields로 지정해줌

In [ ]:
path('articles/<int:article_pk>/comments/', views.comment_create),


@api_view(['POST'])
def comment_create(request, article_pk):
    article = Article.objects.get(pk = article_pk)
    serializer = CommentSerializer(data = request.data)
    if serializer.is_valid(raise_exception=True):
        serializer.save(article=article)
        return Response(serializer.data, status=status.HTTP_201_CREATED)
# raise_exception=True를 통해 실패했을 때 메시지를 생략
# save(article=article)를 해줘서 생성하려는 댓글의 게시물 pk값을 넣어줌
# 하지만 is_valid 이후에 넣어주므로 400 에러는 계속 뜸
# serializer.py 에서 article을 읽기전용 필드로 바꿔줘야 함
    
class CommentSerializer(serializers.ModelSerializer):

    class Meta:
        model = Comment
        fields = '__all__'
        read_only_fields = ('article',)

## 단일 댓글 삭제 및 수정
1. comment_detail view 함수 수정

In [ ]:
# views.py
@api_view(['GET', 'DELETE', 'PUT'])
def comment_detail(request, comment_pk):
    comment = Comment.objects.get(pk = comment_pk)
    if request.method == 'POST':
        serializer = CommentSerializer(comment)
        return Response(serializer.data)
    
    elif request.method == 'DELETE':
        comment.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)
    
    elif request.method == 'PUT':
        serializer = CommentSerializer(comment, data=request.data)
        if serializer.is_valid(raise_exception=True):
            serializer.save()
            return Response(serializer.data)

## 댓글 조회 시 게시글 제목까지 제공하고 싶다면?
- CommentSerializer를 변경

In [ ]:
class CommentSerializer(serializers.ModelSerializer):
    class ArticleTitleSerializer(serializers.ModelSerializer):
        class Meta:
            model = Article
            fields = ('title',)
    article = ArticleTitleSerializer(read_only=True)
    
    class Meta:
        model = Comment
        fields = '__all__'
        read_only_fields = ('article',)

## 단일 게시글 & 댓글 목록
- 역참조를 할때는 참조되는 대상에 포함하여 작성
- 단일 게시글을 조회할 때, 그 게시글에 달린 댓글마저 보고싶다면

In [ ]:
class ArticleSerializer(serializers.ModelSerializer):
    class CommentDetailSerializer(serializers.ModelSerializer):
        class Meta:
            model = Comment
            fields = ('id', 'content',)
    
    comment_set = CommentDetailSerializer(read_only=True, many=True)

    class Meta:
        model = Article
        fields = '__all__'

## 단일 게시글 + 댓글 개수
- 댓글 개수를 표현하는 새로운 필드 생성
- source
  - 필드를 채우는데 사용할 속성의 이름
  - 점 표기법을 사용하여 속성을 탐색 할 수 있음
- 특정 필드를 추가, 수정한 경우 read_only_fields는 동작하지 않음
- 이런 경우 새로운 필드에 read_only 키워드 인자로 작성해야 함

In [ ]:
class ArticleSerializer(serializers.ModelSerializer):
    class CommentDetailSerializer(serializers.ModelSerializer):
        class Meta:
            model = Comment
            fields = ('id', 'content',)
    
    comment_set = CommentDetailSerializer(read_only=True, many=True)
    comment_count = serializers.IntegerField(source='comment_set.count', read_only=True)

## API 문서화

### OAS
- RESTful API를 설명하고 시각화하는 표준화된 방법
- swagger 와 Redoc을 사용
1. drf-spcetacular 설치
2. settings.py에 등록
3. url.py 에 주소 추가

In [ ]:
# settings.py
INSTALLED_APPS = [
    # ALL YOUR APPS
    'drf_spectacular',
]

REST_FRAMEWORK = {
    # YOUR SETTINGS
    'DEFAULT_SCHEMA_CLASS': 'drf_spectacular.openapi.AutoSchema',
}

# urls.py

from drf_spectacular.views import SpectacularAPIView, SpectacularRedocView, SpectacularSwaggerView

# YOUR PATTERNS
    path('api/schema/', SpectacularAPIView.as_view(), name='schema'),
    # Optional UI:
    path('api/schema/swagger-ui/', SpectacularSwaggerView.as_view(url_name='schema'), name='swagger-ui'),
    path('api/schema/redoc/', SpectacularRedocView.as_view(url_name='schema'), name='redoc'),

## get_object_or_404() get_list_or_404()
- 500에러 대신 404에러로 반환시켜주는 숏컷 함수
- 단일 데이터 조회시에는 object, 다중 데이터 조회시에는 list를 사용

In [ ]:
 # articles = Article.objects.all()
articles = get_list_or_404(Article)


# article = Article.objects.get(pk=article_pk)
article = get_object_or_404(Article, pk = article_pk)